# Optimization of the Cross Section of a Beam

In [27]:
L = 100.  # length of the console
w, h = 10., 10.  # width and height of the rectangular cross section
F = -1000.  # value of the vertical load at the free end
E = 210000.0  # Young's modulus
nu = 0.3  # Poisson's ratio


In [28]:
from sigmaepsilon.solid import Structure, LineMesh, PointData
from neumann.linalg import linspace, Vector
from neumann.array import repeat
from polymesh.space import StandardFrame, PointCloud
from sigmaepsilon.solid.fem.cells import B2 as Beam

import numpy as np

# space
GlobalFrame = StandardFrame(dim=3)

# mesh
nElem = 2  # number of finite elements to use
p0 = np.array([0., 0., 0.])
p1 = np.array([L, 0., 0.])
coords = linspace(p0, p1, nElem+1)
coords = PointCloud(coords, frame=GlobalFrame).show()
topo = np.zeros((nElem, 2), dtype=int)
topo[:, 0] = np.arange(nElem)
topo[:, 1] = np.arange(nElem) + 1

# support at the leftmost, load at the rightmost node
loads = np.zeros((coords.shape[0], 6))
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
global_load_vector = Vector([0., 0, F], frame=GlobalFrame).show()
loads[-1, :3] = global_load_vector
fixity[0, :] = True

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = repeat(GlobalFrame.show(), topo.shape[0])
cd = Beam(topo=topo, frames=frames)

# set up mesh and structure
def build(model:np.ndarray):
    mesh = LineMesh(pd, cd, model=model, frame=GlobalFrame)
    structure = Structure(mesh=mesh)
    return structure

def linsolve(structure:Structure):
    structure.linear_static_analysis()
    return structure

def postproc(structure:Structure):    
    dofsol = structure.nodal_dof_solution()[:, :3]
    return dofsol[-1, :3]

In [29]:
# cross section
A = w * h  # area
Iy = w * h**3 / 12  # second moment of inertia around the y axis
Iz = h * w**3 / 12  # second moment of inertia around the z axis
Ix = Iy + Iz  # torsional inertia

# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

d = postproc(linsolve(build(Hooke)))

In [30]:
from neumann.optimize import BinaryGeneticAlgorithm

Vmax = w * h * L
Wmax = np.abs(global_load_vector @ d)


def objective_analytic(x):
    w, h = x
    A = w * h  # area
    Iy = w * h**3 / 12  # second moment of inertia around the y axis
    EI = E * Iy
    sol_exact = F * L**3 / (3 * EI)
    res = np.abs(sol_exact * F)
    # check volume constraint
    V = A * L
    if V > Vmax:
        res *= 1e10
    return res
    
    
def objective_fem(x):
    w, h = x
    A = w * h  # area
    Iy = w * h**3 / 12  # second moment of inertia around the y axis
    Iz = w * h**3 / 12  # second moment of inertia around the z axis
    Ix = Iy + Iz  # torsional inertia
    # model stiffness matrix
    G = E / (2 * (1 + nu))
    Hooke = np.array([
        [E*A, 0, 0, 0],
        [0, G*Ix, 0, 0],
        [0, 0, E*Iy, 0],
        [0, 0, 0, E*Iz]
    ])
    d = postproc(linsolve(build(Hooke)))
    res = np.abs(global_load_vector @ d)
    # check volume constraint
    V = A * L
    if V > Vmax:
        res *= 1e10
    return res

In [31]:
ranges = [[5., 15.], [5., 15.]]
BGA = BinaryGeneticAlgorithm(objective_analytic, ranges, length=12, 
                             nPop=100, maxiter=100)
BGA.solve()
w, h = BGA.best_phenotype()
fx_analytic = objective_analytic((w, h))
(w, h), fx_analytic, (w * h * L, Vmax)

((6.6581196581196584, 15.0), 847.6475742201029, (9987.179487179488, 10000.0))

In [32]:
ranges = [[5., 15.], [5., 15.]]
BGA = BinaryGeneticAlgorithm(objective_fem, ranges, length=12, 
                             nPop=100, maxiter=100)
BGA.solve()
w, h = BGA.best_phenotype()
fx_fem = objective_fem((w, h))
(w, h), fx_fem, (w * h * L, Vmax)

((7.456654456654457, 6.028083028083028),
 1904.7719057619004,
 (4494.93321764384, 10000.0))

## Optimization with `SciPy`

In [33]:
from scipy.optimize import minimize

x0 = [8.0, 8.0]
bnds = ((5., 15.), (5., 15.))
res = minimize(objective_analytic, x0, method='Nelder-Mead', bounds=bnds)
w, h = res.x
fx_analytic = objective_analytic((w, h))
(w, h), fx_analytic, (w * h * L, Vmax)

((5.0, 15.0), 1128.747795414462, (7500.0, 10000.0))

In [34]:
x0 = [8.0, 8.0]
bnds = ((5., 15.), (5., 15.))

def objective_analytic(x):
    w, h = x
    Iy = w * h**3 / 12  # second moment of inertia around the y axis
    EI = E * Iy
    sol_exact = F * L**3 / (3 * EI)
    return np.abs(sol_exact * F)

cons = ({'type': 'ineq', 'fun': lambda x:  Vmax - x[0] * x[1] * L})

res = minimize(objective_analytic, x0, method='SLSQP', bounds=bnds, 
               constraints=cons)
w, h = res.x
fx_analytic = objective_analytic((w, h))
(w, h), fx_analytic, (w * h * L, Vmax)

((6.6666666666667425, 15.0), 846.5608465608369, (10000.000000000113, 10000.0))

In [35]:
x0 = [8.0, 8.0]
bnds = ((5., 15.), (5., 15.))

def objective_fem(x):
    w, h = x
    A = w * h  # area
    Iy = w * h**3 / 12  # second moment of inertia around the y axis
    Iz = h * w**3 / 12  # second moment of inertia around the z axis
    Ix = Iy + Iz  # torsional inertia

    # model stiffness matrix
    G = E / (2 * (1 + nu))
    Hooke = np.array([
        [E*A, 0, 0, 0],
        [0, G*Ix, 0, 0],
        [0, 0, E*Iy, 0],
        [0, 0, 0, E*Iz]
    ])

    d = postproc(linsolve(build(Hooke)))
    return np.abs(global_load_vector @ d)
    
cons = ({'type': 'ineq', 'fun': lambda x:  Vmax - x[0] * x[1] * L})

res = minimize(objective_fem, x0, method='SLSQP', bounds=bnds, 
               constraints=cons)
w, h = res.x
fx_fem = objective_fem((w, h))
(w, h), fx_fem, (w * h * L, Vmax)

((8.0, 8.0), 1904.7719057619004, (6400.0, 10000.0))